In [ ]:
class Technique(ABC):
    dataset = None
    method = None
    methodName = None
    test_size = None
    mm_scaler = None
    
    @abstractmethod 
    def loadCurrentMethod(Self,methodName,dataset):
         pass
    @abstractmethod 
    def train(Self):
         pass
    @abstractmethod 
    def test(Self,newX,newY=None):
         pass
        
    def getMethodName(Self):
        return Self.methodName
    
    @abstractmethod 
    def score(Self,y_test,y_prediction):
        pass
    
    def featureSelection(dataset,option=1,featuresToRemove=None):
        if(option==1):
            # remove features
            return dataset.drop(featuresToRemove, axis=1)
        return None
    def normalizeTrainData(Self,option=1):
        Self.mm_scaler = None
        if(option==1):
            Self.mm_scaler = preprocessing.MinMaxScaler()
            Self.dataset.x_train = Self.mm_scaler.fit_transform(Self.dataset.x_train)
            
    def normalizeTestData(Self,data):
        if(Self.mm_scaler!=None):
            return Self.mm_scaler.transform(data)
            

In [ ]:
class Regression(Technique):
    def loadCurrentMethod(self,methodName,dataset):
        self.methodName = methodName
        self.dataset = dataset
        if(methodName== 'Lasso'):
            self.method = linear_model.Lasso(alpha=0.1)
            return self.method 
        if(methodName== 'ElasticNet'):
            self.method = linear_model.ElasticNet(random_state=0)
            return self.method 
        if(methodName== 'RidgeRegression'):
            self.method = linear_model.Ridge(alpha=.5)
            return self.method 
        
        raise ValueError('Method not found','Regression-getMethod')
        
    def score(Self,y_actual,y_prediction):
        score=r2_score(y_actual,y_prediction)
        square = mean_squared_error(y_actual,y_prediction)
        rootsquare = np.sqrt(mean_squared_error(y_actual,y_prediction))
        print('r2 score is ',score)
        print('mean_sqrd_error is==',square)
        print('root_mean_squared error of is=',rootsquare)
        return score
    
    def train(self):
        self.normalizeTrainData()
        self.method.fit(self.dataset.x_train,self.dataset.y_train)
        
        self.dataset.x_validate = self.normalizeTestData(data=self.dataset.x_validate)
        y_prediction =  self.method.predict(self.dataset.x_validate)
        score = self.score(self.dataset.y_validate,y_prediction)
        return y_prediction, score
#         r2_score = self.method.score(self.dataset.x_validate, self.dataset.y_validate)
#         print(r2_score)
        
#         cross_validate_scores = cross_val_score(self.method, self.dataset.x_train, self.dataset.y_train, cv=5)
#         print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_validate_scores.mean(), cross_validate_scores.std()))
#         print("accuracy across K folds ", cross_validate_scores)
        
        
    def test(self,newX,newY=None):
       # newX = self.normalizeData(newX)
        y_prediction =  self.method.predict(newX)
        score = self.score(newY,y_prediction)
        return y_prediction,score
        
        

In [ ]:
# Class Classification(Technique):
#     # classification metrics 
#     scoring = ['precision_macro', 'recall_macro']
# #precision - dentro do que eu retornei, quantos foram assertivos?
#         #recall - considerando todas as possibilidades corretas que eu poderia retornar, quantos eu encontrei?  
#         scoring = ['precision_macro', 'recall_macro']
#         scores = cross_validate(self.method, self.dataset.x_train, self.dataset.y_train, scoring=scoring)
#         sorted(scores.keys())
#         #cross_validate = cross_val_score(self.method, X, y, cv=3)